In [1]:
# reference: http://cs229.stanford.edu/notes/cs229-notes1.pdf
import numpy as np
import matplotlib as ml
import matplotlib.pyplot as plt
import random
import sys

data = np.loadtxt('assign1_data.txt')
#print data

x1 = data[:, 0]
print ("x1:", x1)

x2 = data[:, 1]
print ("x2:", x2)

y = data[:, 2]
#print ("y:", y)
'''
solution from part1 is:
w1:	-2.04424259514
w2:	3.99686016866
b:	-0.924290811868
'''

('x1:', array([ 0.1227,  0.3914,  0.7725,  0.8342,  0.5084,  0.9983,  0.6643,
        0.0954,  0.6949,  0.6075,  0.3717,  0.0193,  0.9314,  0.2206,
        0.0903,  0.5466,  0.4331,  0.5293,  0.8955,  0.0701,  0.0771,
        0.1972,  0.6405,  0.3203,  0.103 ,  0.5605,  0.4288,  0.7101,
        0.6575,  0.9498,  0.555 ,  0.0947,  0.4293,  0.31  ,  0.7297,
        0.1973,  0.1571,  0.2272,  0.2099,  0.3972,  0.0033,  0.6441,
        0.5549,  0.9193,  0.5341,  0.1555,  0.1839,  0.9949,  0.7918,
        0.4088,  0.2247,  0.3901,  0.4309,  0.0732,  0.8077,  0.0854,
        0.8955,  0.8794,  0.1094,  0.0726,  0.4377,  0.889 ,  0.5791,
        0.9721,  0.5719,  0.3156,  0.8065,  0.4777,  0.6264,  0.2117,
        0.042 ,  0.0916,  0.6035,  0.6354,  0.4102,  0.695 ,  0.2153,
        0.0768,  0.2356,  0.1591,  0.7171,  0.9988,  0.3936,  0.0201,
        0.6295,  0.1771,  0.5888,  0.5302,  0.3434,  0.7154,  0.3506,
        0.5432,  0.4687,  0.013 ,  0.1311,  0.697 ,  0.3996,  0.2208,
        0.11

'\nsolution from part1 is:\nw1:\t-2.04424259514\nw2:\t3.99686016866\nb:\t-0.924290811868\n'

In [18]:
def plot_error_epoch(errors, lr, batch_size):
    errors_y = np.array(errors)
    epochs = []
    for i in range(len(errors)):
        epochs.append(i)
    epochs_x = np.array(epochs)
    plt.plot(epochs_x, errors_y)
    title = ''
    title += 'lr:'
    title += str(lr)
    title += ' batch_size:'
    title += str(batch_size)
    plt.title(title)
    plt.xlabel('epoch')
    plt.ylabel('error')
    plt.show()

In [16]:
# create method to train and get result with different settings like online, minibatch, batch and learning rate
def train_and_get_result(x1, x2, lr, batch_size):
    
    if batch_size > 75:
        batch_size = 75
        
    errors = []
    
    w1 = random.random() * 3
    w2 = random.random() * 3
    b = random.random() * 3
    
    print "random initial w1:\t", w1
    print "random initial w2:\t", w2
    print "random initial b:\t", b
    
    epoch = 0
    # training set size : validation set size = 3 : 1
    training_set_index = 0
    size = len(x1)
    validation_set_index = 0
    validation_set_index_step = size / 4
    pre_validation_error = sys.maxint
    curr_validation_error = sys.maxint - 1
    while curr_validation_error >= 1 and (pre_validation_error >= curr_validation_error or curr_validation_error <= 0): # early stopping when the validation error is greater than its previous one
    #while True:

        validation_set_index += validation_set_index_step
        validation_set_index %= size

        x1_validation = x1[validation_set_index : validation_set_index + validation_set_index_step]
        x2_validation = x2[validation_set_index : validation_set_index + validation_set_index_step]
        y_validation = y[validation_set_index : validation_set_index + validation_set_index_step]

        # training
        i = 1
        for x1i, x2i, yi in zip(x1, x2, y):
            sum_error_w1 = 0
            sum_error_w2 = 0
            sum_error_b = 0
            if x1i not in zip(x1_validation) and x2i not in zip(x2_validation) and yi not in zip(y_validation):
                error = yi - (x1i * w1 + x2i * w2 + b)
                sum_error_w1 += error * x1i
                sum_error_w2 += error * x2i
                sum_error_b += error
                if i >= batch_size: # only update weights after one batch
                    w1 = w1 + lr * sum_error_w1
                    w2 = w2 + lr * sum_error_w2
                    b = b + lr * sum_error_b
                    sum_error_w1 = 0
                    sum_error_w2 = 0
                    sum_error_b = 0
                    i = 1
                    continue
                i += 1

        if epoch >= 4 and epoch % 4 == 0:
            pre_validation_error = curr_validation_error
            if epoch % 100 == 0: # print only every 20 epochs
                print "epoch:\t", epoch, "\t curr_validation_error:\t", curr_validation_error
            curr_validation_error = 0

        # validation
        for x1i, x2i, yi in zip(x1_validation, x2_validation, y_validation):
            curr_validation_error += abs(yi - (x1i * w1 + x2i * w2 + b))
            
        # get error from the entire set for later plotting
        error = 0
        for x1i, x2i, yi in zip(x1, x2, y):
            error += yi - (x1i * w1 + x2i * w2 + b)
        errors.append(abs(error))
        epoch += 1

    plot_error_epoch(errors, lr, batch_size)
    print "Result: pre_validation_error:\t", pre_validation_error, "curr_validation_error:\t", curr_validation_error
    print "Result: w1:\t", w1, "\tw2:\t", w2, "\tb:\t", b, "\tnum of epoch:\t", epoch

In [17]:
# online
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 1)

random initial w1:	0.0753723114524
random initial w2:	0.476640634316
random initial b:	1.51594931515
epoch:	100 	 curr_validation_error:	100.955851835
epoch:	200 	 curr_validation_error:	90.4474350483
epoch:	300 	 curr_validation_error:	85.6795977763
epoch:	400 	 curr_validation_error:	81.821730402
epoch:	500 	 curr_validation_error:	78.0765813239
epoch:	600 	 curr_validation_error:	74.4981068449
epoch:	700 	 curr_validation_error:	71.0672845106
epoch:	800 	 curr_validation_error:	67.791849085
epoch:	900 	 curr_validation_error:	64.66946014
epoch:	1000 	 curr_validation_error:	61.6945580465
epoch:	1100 	 curr_validation_error:	58.8607058384
epoch:	1200 	 curr_validation_error:	56.1613821342
epoch:	1300 	 curr_validation_error:	53.6016644308
epoch:	1400 	 curr_validation_error:	51.1791881457
epoch:	1500 	 curr_validation_error:	48.8858113629
epoch:	1600 	 curr_validation_error:	46.7348365255
epoch:	1700 	 curr_validation_error:	44.6940413804
epoch:	1800 	 curr_validation_error:	42.76488

In [ ]:
# online
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 1)

In [11]:
# online
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 1)

random initial w1:	0.0221804168987
random initial w2:	1.91608356896
random initial b:	2.20818035244
errors: [-275.06169566410989]
errors: [-275.06169566410989, -247.87663723633503]
errors: [-275.06169566410989, -247.87663723633503, -222.94446196429789]
errors: [-275.06169566410989, -247.87663723633503, -222.94446196429789, -201.20402848609334]
errors: [-275.06169566410989, -247.87663723633503, -222.94446196429789, -201.20402848609334, -181.27973427246997]
errors: [-275.06169566410989, -247.87663723633503, -222.94446196429789, -201.20402848609334, -181.27973427246997, -163.52523315366608]
errors: [-275.06169566410989, -247.87663723633503, -222.94446196429789, -201.20402848609334, -181.27973427246997, -163.52523315366608, -147.27859435802515]
errors: [-275.06169566410989, -247.87663723633503, -222.94446196429789, -201.20402848609334, -181.27973427246997, -163.52523315366608, -147.27859435802515, -133.05026419840266]
errors: [-275.06169566410989, -247.87663723633503, -222.94446196429789, 

In [16]:
# minibatch batch_size = 50
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 1, batch_size = 1)

random initial w1:	 2.638794869127527
random initial w2:	 0.5597741066467697
random initial b:	 1.7493370997628133


TypeError: slice indices must be integers or None or have an __index__ method

In [6]:
# online
# learning rate lr = 0.005
train_and_get_result(x1, x2, lr = 0.005, batch_size = 1)

random initial w1:	0.829831210856
random initial w2:	1.90464819925
random initial b:	2.84259493978
epoch:	100 	 curr_validation_error:	17.6146627327
epoch:	200 	 curr_validation_error:	16.22914788
Result: pre_validation_error:	16.2278047701 curr_validation_error:	16.2293951512
Result: w1:	-2.04257641852 	w2:	3.98465541592 	b:	-0.917493466752 	num of epoch:	224


In [ ]:
# online
# learning rate lr = 0.01
train_and_get_result(x1, x2, lr = 1, batch_size = 1)

random initial w1:	0.397166181774
random initial w2:	0.202937542594
random initial b:	2.20915997125
epoch:	100 	 curr_validation_error:	37.2192177706
epoch:	200 	 curr_validation_error:	37.2192177706
epoch:	300 	 curr_validation_error:	37.2192177706
epoch:	400 	 curr_validation_error:	37.2192177706
epoch:	500 	 curr_validation_error:	37.2192177706
epoch:	600 	 curr_validation_error:	37.2192177706
epoch:	700 	 curr_validation_error:	37.2192177706
epoch:	800 	 curr_validation_error:	37.2192177706
epoch:	900 	 curr_validation_error:	37.2192177706
epoch:	1000 	 curr_validation_error:	37.2192177706
epoch:	1100 	 curr_validation_error:	37.2192177706
epoch:	1200 	 curr_validation_error:	37.2192177706
epoch:	1300 	 curr_validation_error:	37.2192177706
epoch:	1400 	 curr_validation_error:	37.2192177706
epoch:	1500 	 curr_validation_error:	37.2192177706
epoch:	1600 	 curr_validation_error:	37.2192177706
epoch:	1700 	 curr_validation_error:	37.2192177706
epoch:	1800 	 curr_validation_error:	37.21

epoch:	16300 	 curr_validation_error:	37.2192177706
epoch:	16400 	 curr_validation_error:	37.2192177706
epoch:	16500 	 curr_validation_error:	37.2192177706
epoch:	16600 	 curr_validation_error:	37.2192177706
epoch:	16700 	 curr_validation_error:	37.2192177706
epoch:	16800 	 curr_validation_error:	37.2192177706
epoch:	16900 	 curr_validation_error:	37.2192177706
epoch:	17000 	 curr_validation_error:	37.2192177706
epoch:	17100 	 curr_validation_error:	37.2192177706
epoch:	17200 	 curr_validation_error:	37.2192177706
epoch:	17300 	 curr_validation_error:	37.2192177706
epoch:	17400 	 curr_validation_error:	37.2192177706
epoch:	17500 	 curr_validation_error:	37.2192177706
epoch:	17600 	 curr_validation_error:	37.2192177706
epoch:	17700 	 curr_validation_error:	37.2192177706
epoch:	17800 	 curr_validation_error:	37.2192177706
epoch:	17900 	 curr_validation_error:	37.2192177706
epoch:	18000 	 curr_validation_error:	37.2192177706
epoch:	18100 	 curr_validation_error:	37.2192177706
epoch:	18200

epoch:	33500 	 curr_validation_error:	37.2192177706
epoch:	33600 	 curr_validation_error:	37.2192177706
epoch:	33700 	 curr_validation_error:	37.2192177706
epoch:	33800 	 curr_validation_error:	37.2192177706
epoch:	33900 	 curr_validation_error:	37.2192177706
epoch:	34000 	 curr_validation_error:	37.2192177706
epoch:	34100 	 curr_validation_error:	37.2192177706
epoch:	34200 	 curr_validation_error:	37.2192177706
epoch:	34300 	 curr_validation_error:	37.2192177706
epoch:	34400 	 curr_validation_error:	37.2192177706
epoch:	34500 	 curr_validation_error:	37.2192177706
epoch:	34600 	 curr_validation_error:	37.2192177706
epoch:	34700 	 curr_validation_error:	37.2192177706
epoch:	34800 	 curr_validation_error:	37.2192177706
epoch:	34900 	 curr_validation_error:	37.2192177706
epoch:	35000 	 curr_validation_error:	37.2192177706
epoch:	35100 	 curr_validation_error:	37.2192177706
epoch:	35200 	 curr_validation_error:	37.2192177706
epoch:	35300 	 curr_validation_error:	37.2192177706
epoch:	35400

epoch:	49600 	 curr_validation_error:	37.2192177706
epoch:	49700 	 curr_validation_error:	37.2192177706
epoch:	49800 	 curr_validation_error:	37.2192177706
epoch:	49900 	 curr_validation_error:	37.2192177706
epoch:	50000 	 curr_validation_error:	37.2192177706
epoch:	50100 	 curr_validation_error:	37.2192177706
epoch:	50200 	 curr_validation_error:	37.2192177706
epoch:	50300 	 curr_validation_error:	37.2192177706
epoch:	50400 	 curr_validation_error:	37.2192177706
epoch:	50500 	 curr_validation_error:	37.2192177706
epoch:	50600 	 curr_validation_error:	37.2192177706
epoch:	50700 	 curr_validation_error:	37.2192177706
epoch:	50800 	 curr_validation_error:	37.2192177706
epoch:	50900 	 curr_validation_error:	37.2192177706
epoch:	51000 	 curr_validation_error:	37.2192177706
epoch:	51100 	 curr_validation_error:	37.2192177706
epoch:	51200 	 curr_validation_error:	37.2192177706
epoch:	51300 	 curr_validation_error:	37.2192177706
epoch:	51400 	 curr_validation_error:	37.2192177706
epoch:	51500

In [8]:
# minibatch batch_size = 5
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 5)

random initial w1:	0.935800006705
random initial w2:	0.762838865253
random initial b:	2.70462440369
epoch:	100 	 curr_validation_error:	281.401378624
epoch:	200 	 curr_validation_error:	232.984358616
epoch:	300 	 curr_validation_error:	194.987043556
epoch:	400 	 curr_validation_error:	164.372736828
epoch:	500 	 curr_validation_error:	141.996682255
epoch:	600 	 curr_validation_error:	125.543455704
epoch:	700 	 curr_validation_error:	114.472527585
epoch:	800 	 curr_validation_error:	106.942904348
epoch:	900 	 curr_validation_error:	102.229628594
epoch:	1000 	 curr_validation_error:	98.9465819837
epoch:	1100 	 curr_validation_error:	96.5610068305
epoch:	1200 	 curr_validation_error:	94.9869306765
epoch:	1300 	 curr_validation_error:	93.655436972
epoch:	1400 	 curr_validation_error:	92.4442440421
epoch:	1500 	 curr_validation_error:	91.4201248048
epoch:	1600 	 curr_validation_error:	90.4305787227
epoch:	1700 	 curr_validation_error:	89.5674419736
epoch:	1800 	 curr_validation_error:	88.758

epoch:	15900 	 curr_validation_error:	26.3965069397
epoch:	16000 	 curr_validation_error:	26.2230475202
epoch:	16100 	 curr_validation_error:	26.0512970993
epoch:	16200 	 curr_validation_error:	25.8812387183
epoch:	16300 	 curr_validation_error:	25.7128555876
epoch:	16400 	 curr_validation_error:	25.5461310853
epoch:	16500 	 curr_validation_error:	25.381048755
epoch:	16600 	 curr_validation_error:	25.2175923046
epoch:	16700 	 curr_validation_error:	25.0557456043
epoch:	16800 	 curr_validation_error:	24.8954926855
epoch:	16900 	 curr_validation_error:	24.7368177386
epoch:	17000 	 curr_validation_error:	24.5797051116
epoch:	17100 	 curr_validation_error:	24.4249354971
epoch:	17200 	 curr_validation_error:	24.2753507737
epoch:	17300 	 curr_validation_error:	24.1272355916
epoch:	17400 	 curr_validation_error:	23.981679447
epoch:	17500 	 curr_validation_error:	23.8384246163
epoch:	17600 	 curr_validation_error:	23.6965808304
epoch:	17700 	 curr_validation_error:	23.5561340899
epoch:	17800 	

epoch:	31700 	 curr_validation_error:	16.7700468218
epoch:	31800 	 curr_validation_error:	16.7575573022
epoch:	31900 	 curr_validation_error:	16.74536733
epoch:	32000 	 curr_validation_error:	16.7332986066
epoch:	32100 	 curr_validation_error:	16.7213499203
epoch:	32200 	 curr_validation_error:	16.709520072
epoch:	32300 	 curr_validation_error:	16.6978078742
epoch:	32400 	 curr_validation_error:	16.6862121516
epoch:	32500 	 curr_validation_error:	16.6747317405
epoch:	32600 	 curr_validation_error:	16.663365489
epoch:	32700 	 curr_validation_error:	16.6521122567
epoch:	32800 	 curr_validation_error:	16.6409709144
epoch:	32900 	 curr_validation_error:	16.6299403447
epoch:	33000 	 curr_validation_error:	16.6190194409
epoch:	33100 	 curr_validation_error:	16.6085139982
epoch:	33200 	 curr_validation_error:	16.5988828701
epoch:	33300 	 curr_validation_error:	16.5904810268
epoch:	33400 	 curr_validation_error:	16.5821639138
epoch:	33500 	 curr_validation_error:	16.5739306762
epoch:	33600 	 c

epoch:	47500 	 curr_validation_error:	16.3004784657
epoch:	47600 	 curr_validation_error:	16.3004293568
epoch:	47700 	 curr_validation_error:	16.3003825215
epoch:	47800 	 curr_validation_error:	16.3003379244
epoch:	47900 	 curr_validation_error:	16.3002955304
epoch:	48000 	 curr_validation_error:	16.3002553049
epoch:	48100 	 curr_validation_error:	16.3002172136
epoch:	48200 	 curr_validation_error:	16.3001812228
epoch:	48300 	 curr_validation_error:	16.3001472992
epoch:	48400 	 curr_validation_error:	16.3001154099
epoch:	48500 	 curr_validation_error:	16.3000855223
epoch:	48600 	 curr_validation_error:	16.3000576044
epoch:	48700 	 curr_validation_error:	16.3000316245
epoch:	48800 	 curr_validation_error:	16.3000075514
epoch:	48900 	 curr_validation_error:	16.2999853542
epoch:	49000 	 curr_validation_error:	16.2999650023
epoch:	49100 	 curr_validation_error:	16.2999464658
epoch:	49200 	 curr_validation_error:	16.2999297149
epoch:	49300 	 curr_validation_error:	16.2999147203
epoch:	49400

In [9]:
# minibatch batch_size = 25
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 25)

random initial w1:	1.03416040962
random initial w2:	0.890557363878
random initial b:	2.66951474359
epoch:	100 	 curr_validation_error:	336.185133692
epoch:	200 	 curr_validation_error:	324.707664116
epoch:	300 	 curr_validation_error:	313.570875465
epoch:	400 	 curr_validation_error:	302.764715821
epoch:	500 	 curr_validation_error:	292.279429767
epoch:	600 	 curr_validation_error:	282.105549643
epoch:	700 	 curr_validation_error:	272.233887057
epoch:	800 	 curr_validation_error:	262.658489458
epoch:	900 	 curr_validation_error:	253.545005942
epoch:	1000 	 curr_validation_error:	244.82532534
epoch:	1100 	 curr_validation_error:	236.420131958
epoch:	1200 	 curr_validation_error:	228.422796518
epoch:	1300 	 curr_validation_error:	220.662704919
epoch:	1400 	 curr_validation_error:	213.248914809
epoch:	1500 	 curr_validation_error:	206.104437399
epoch:	1600 	 curr_validation_error:	199.171846514
epoch:	1700 	 curr_validation_error:	192.44489062
epoch:	1800 	 curr_validation_error:	185.9175

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 50)

random initial w1:	0.539547051629
random initial w2:	1.2966094591
random initial b:	2.11174909945
epoch:	100 	 curr_validation_error:	284.786066027
epoch:	200 	 curr_validation_error:	280.079704167
epoch:	300 	 curr_validation_error:	275.43550138
epoch:	400 	 curr_validation_error:	270.852645799
epoch:	500 	 curr_validation_error:	266.330336155
epoch:	600 	 curr_validation_error:	261.86778164
epoch:	700 	 curr_validation_error:	257.464201767
epoch:	800 	 curr_validation_error:	253.11882624
epoch:	900 	 curr_validation_error:	248.830894818
epoch:	1000 	 curr_validation_error:	244.599657182
epoch:	1100 	 curr_validation_error:	240.424372811
epoch:	1200 	 curr_validation_error:	236.304310849
epoch:	1300 	 curr_validation_error:	232.238749982
epoch:	1400 	 curr_validation_error:	228.226978309
epoch:	1500 	 curr_validation_error:	224.268293227
epoch:	1600 	 curr_validation_error:	220.378516669
epoch:	1700 	 curr_validation_error:	216.594303947
epoch:	1800 	 curr_validation_error:	212.860155

In [ ]:
# minibatch batch_size = 5
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 5)

random initial w1:	1.13565797461
random initial w2:	0.976967562815
random initial b:	0.286890705608
epoch:	100 	 curr_validation_error:	96.5840036928
epoch:	200 	 curr_validation_error:	87.0876301133
epoch:	300 	 curr_validation_error:	82.4713563233
epoch:	400 	 curr_validation_error:	78.5301883403
epoch:	500 	 curr_validation_error:	74.8825338749
epoch:	600 	 curr_validation_error:	71.3885044696
epoch:	700 	 curr_validation_error:	68.0727536387
epoch:	800 	 curr_validation_error:	64.9259888915
epoch:	900 	 curr_validation_error:	61.9520431091
epoch:	1000 	 curr_validation_error:	59.1330877187
epoch:	1100 	 curr_validation_error:	56.4624960657
epoch:	1200 	 curr_validation_error:	53.9250214933
epoch:	1300 	 curr_validation_error:	51.5333971994
epoch:	1400 	 curr_validation_error:	49.2900887405
epoch:	1500 	 curr_validation_error:	47.15728733
epoch:	1600 	 curr_validation_error:	45.1295485635
epoch:	1700 	 curr_validation_error:	43.2016920278
epoch:	1800 	 curr_validation_error:	41.3949

epoch:	15900 	 curr_validation_error:	16.3420535937
epoch:	16000 	 curr_validation_error:	16.3419991161
epoch:	16100 	 curr_validation_error:	16.3419455751
epoch:	16200 	 curr_validation_error:	16.3418929856
epoch:	16300 	 curr_validation_error:	16.3418413596
epoch:	16400 	 curr_validation_error:	16.3417907066
epoch:	16500 	 curr_validation_error:	16.3417410336
epoch:	16600 	 curr_validation_error:	16.3416923451
epoch:	16700 	 curr_validation_error:	16.3416446437
epoch:	16800 	 curr_validation_error:	16.3415979303
epoch:	16900 	 curr_validation_error:	16.3415522036
epoch:	17000 	 curr_validation_error:	16.341507461
epoch:	17100 	 curr_validation_error:	16.3414636983
epoch:	17200 	 curr_validation_error:	16.3414209101
epoch:	17300 	 curr_validation_error:	16.3413790895
epoch:	17400 	 curr_validation_error:	16.3413382288
epoch:	17500 	 curr_validation_error:	16.3412983191
epoch:	17600 	 curr_validation_error:	16.3412593506
epoch:	17700 	 curr_validation_error:	16.3412213128
epoch:	17800 

epoch:	32000 	 curr_validation_error:	16.3399429395
epoch:	32100 	 curr_validation_error:	16.3399425591
epoch:	32200 	 curr_validation_error:	16.3399421916
epoch:	32300 	 curr_validation_error:	16.3399418367
epoch:	32400 	 curr_validation_error:	16.3399414938
epoch:	32500 	 curr_validation_error:	16.3399411627
epoch:	32600 	 curr_validation_error:	16.3399408429
epoch:	32700 	 curr_validation_error:	16.339940534
epoch:	32800 	 curr_validation_error:	16.3399402357
epoch:	32900 	 curr_validation_error:	16.3399399475
epoch:	33000 	 curr_validation_error:	16.3399396692
epoch:	33100 	 curr_validation_error:	16.3399394005
epoch:	33200 	 curr_validation_error:	16.3399391409
epoch:	33300 	 curr_validation_error:	16.3399388902
epoch:	33400 	 curr_validation_error:	16.3399386481
epoch:	33500 	 curr_validation_error:	16.3399384143
epoch:	33600 	 curr_validation_error:	16.3399381885
epoch:	33700 	 curr_validation_error:	16.3399379704
epoch:	33800 	 curr_validation_error:	16.3399377598
epoch:	33900 

epoch:	49200 	 curr_validation_error:	16.3399318805
epoch:	49300 	 curr_validation_error:	16.3399318796
epoch:	49400 	 curr_validation_error:	16.3399318787
epoch:	49500 	 curr_validation_error:	16.3399318779
epoch:	49600 	 curr_validation_error:	16.3399318771
epoch:	49700 	 curr_validation_error:	16.3399318763
epoch:	49800 	 curr_validation_error:	16.3399318756
epoch:	49900 	 curr_validation_error:	16.3399318748
epoch:	50000 	 curr_validation_error:	16.3399318741
epoch:	50100 	 curr_validation_error:	16.3399318735
epoch:	50200 	 curr_validation_error:	16.3399318728
epoch:	50300 	 curr_validation_error:	16.3399318722
epoch:	50400 	 curr_validation_error:	16.3399318716
epoch:	50500 	 curr_validation_error:	16.339931871
epoch:	50600 	 curr_validation_error:	16.3399318704
epoch:	50700 	 curr_validation_error:	16.3399318699
epoch:	50800 	 curr_validation_error:	16.3399318693
epoch:	50900 	 curr_validation_error:	16.3399318688
epoch:	51000 	 curr_validation_error:	16.3399318683
epoch:	51100 

epoch:	65200 	 curr_validation_error:	16.3399318548
epoch:	65300 	 curr_validation_error:	16.3399318548
epoch:	65400 	 curr_validation_error:	16.3399318548
epoch:	65500 	 curr_validation_error:	16.3399318548
epoch:	65600 	 curr_validation_error:	16.3399318548
epoch:	65700 	 curr_validation_error:	16.3399318548
epoch:	65800 	 curr_validation_error:	16.3399318548
epoch:	65900 	 curr_validation_error:	16.3399318548
epoch:	66000 	 curr_validation_error:	16.3399318548
epoch:	66100 	 curr_validation_error:	16.3399318548
epoch:	66200 	 curr_validation_error:	16.3399318548
epoch:	66300 	 curr_validation_error:	16.3399318548
epoch:	66400 	 curr_validation_error:	16.3399318548
epoch:	66500 	 curr_validation_error:	16.3399318548
epoch:	66600 	 curr_validation_error:	16.3399318548
epoch:	66700 	 curr_validation_error:	16.3399318548
epoch:	66800 	 curr_validation_error:	16.3399318548
epoch:	66900 	 curr_validation_error:	16.3399318548
epoch:	67000 	 curr_validation_error:	16.3399318548
epoch:	67100

KeyboardInterrupt: 

In [ ]:
# minibatch batch_size = 25
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 25)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 50)

In [ ]:
# minibatch batch_size = 5
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 5)

In [ ]:
# minibatch batch_size = 25
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 25)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 50)

In [ ]:
# minibatch batch_size = 5
# learning rate lr = 0.005
train_and_get_result(x1, x2, lr = 0.005, batch_size = 5)

In [ ]:
# minibatch batch_size = 25
# learning rate lr = 0.005
train_and_get_result(x1, x2, lr = 0.005, batch_size = 25)

In [ ]:
# minibatch batch_size = 50
# learning rate lr = 0.005
train_and_get_result(x1, x2, lr = 0.005, batch_size = 50)

In [ ]:
# batch
# learning rate lr = 0.0001
train_and_get_result(x1, x2, lr = 0.0001, batch_size = 100)

In [ ]:
# batch
# learning rate lr = 0.0005
train_and_get_result(x1, x2, lr = 0.0005, batch_size = 100)

In [ ]:
# batch
# learning rate lr = 0.001
train_and_get_result(x1, x2, lr = 0.001, batch_size = 100)

In [ ]:
# batch
# learning rate lr = 0.005
train_and_get_result(x1, x2, lr = 0.005, batch_size = 100)